# Analysis of Simple Parallel Paths

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from LoadBalancingAnalysis import utils

In [2]:
dir_path = '/Users/mattb/Desktop/Research/ns3-load-balancing/outputs/simple-parallel-paths/0.5'

def construct_lb_path(scheme):
    return os.path.join(os.path.join(dir_path, scheme), 'lb-metrics.csv')

In [4]:
df_ecmp = utils.get_lb_performance_metrics(construct_lb_path('ecmp'))
df_spray = utils.get_lb_performance_metrics(construct_lb_path('packet_spray'))
df_drill = utils.get_lb_performance_metrics(construct_lb_path('drill'))
df_letflow = utils.get_lb_performance_metrics(construct_lb_path('letflow'))

In [9]:
df_ecmp_orig = df_ecmp[df_ecmp["SourceAddress"] == "10.1.1.1"]
df_ecmp_orig_small = df_ecmp_orig[df_ecmp_orig['TimeFirstTxPacket'] < 2000000.0]
df_ecmp_orig_large = df_ecmp_orig[df_ecmp_orig['TimeFirstTxPacket'] >= 2000000.0]

df_spray_orig = df_spray[df_spray["SourceAddress"] == "10.1.1.1"]
df_spray_orig_small = df_spray_orig[df_spray_orig['TimeFirstTxPacket'] < 2000000.0]
df_spray_orig_large = df_spray_orig[df_spray_orig['TimeFirstTxPacket'] >= 2000000.0]

df_drill_orig = df_drill[df_drill["SourceAddress"] == "10.1.1.1"]
df_drill_orig_small = df_drill_orig[df_drill_orig['TimeFirstTxPacket'] < 2000000.0]
df_drill_orig_large = df_drill_orig[df_drill_orig['TimeFirstTxPacket'] >= 2000000.0]

df_letflow_orig = df_letflow[df_letflow["SourceAddress"] == "10.1.1.1"]
df_letflow_orig_small = df_letflow_orig[df_letflow_orig['TimeFirstTxPacket'] < 2000000.0]
df_letflow_orig_large = df_letflow_orig[df_letflow_orig['TimeFirstTxPacket'] >= 2000000.0]

In [10]:
df_ecmp_orig_large

,FlowId,SourceAddress,DestinationAddress,TimeFirstTxPacket,TimeLastTxPacket,TimeFirstRxPacket,TimeLastRxPacket,DelaySum,JitterSum,TxBytes,RxBytes,TxPackets,RxPackets,Duration,EffectiveRate
40,41,10.1.1.1,10.1.4.2,2000000.0,2993720.0,2007200.0,3049600.0,42255800.0,1274050.0,333572,324960,596,581,1049600.0,2476830.0


In [11]:
def print_effective_and_loss_rates(df, scheme):
    effective_rate = df['EffectiveRate'] / 1000000.0
    loss_rate = (df['TxPackets'] - df['RxPackets']) / df['TxPackets']
    print("{0} - effective rate: {1}, loss rate: {2}".format(
        scheme, np.mean(effective_rate), np.mean(loss_rate)))

In [12]:
print("Large Flow Rates")
print("================")
print_effective_and_loss_rates(df_spray_orig_large, "Packet Spray")
print_effective_and_loss_rates(df_ecmp_orig_large, "ECMP")
print_effective_and_loss_rates(df_drill_orig_large, "DRILL")
print_effective_and_loss_rates(df_letflow_orig_large, "LetFlow")

Large Flow Rates
Packet Spray - effective rate: 3.70141, loss rate: 0.0
ECMP - effective rate: 2.47683, loss rate: 0.025167785234899327
DRILL - effective rate: 6.35663, loss rate: 0.0
LetFlow - effective rate: 5.37462, loss rate: 0.0


In [13]:
print("Small Flow Rates")
print("================")
print_effective_and_loss_rates(df_spray_orig_small, "Packet Spray")
print_effective_and_loss_rates(df_ecmp_orig_small, "ECMP")
print_effective_and_loss_rates(df_drill_orig_small, "DRILL")
print_effective_and_loss_rates(df_letflow_orig_small, "LetFlow")

Small Flow Rates
Packet Spray - effective rate: 1.1069085, loss rate: 0.0
ECMP - effective rate: 1.0988159999999998, loss rate: 0.0
DRILL - effective rate: 1.1072885000000001, loss rate: 0.0
LetFlow - effective rate: 1.106585, loss rate: 0.0


In [14]:
def print_delay_and_jitter_sums(df, scheme):
    delay_sum = df['DelaySum'] / 1000000.0
    jitter_sum = df['JitterSum'] / 1000000.0
    print("{0} - delay sum: {1}, jitter sum: {2}".format(
        scheme, np.mean(delay_sum), np.mean(jitter_sum)))

In [15]:
print("Large Flow Sums")
print("================")
print_delay_and_jitter_sums(df_spray_orig_large, "Packet Spray")
print_delay_and_jitter_sums(df_ecmp_orig_large, "ECMP")
print_delay_and_jitter_sums(df_drill_orig_large, "DRILL")
print_delay_and_jitter_sums(df_letflow_orig_large, "LetFlow")

Large Flow Sums
Packet Spray - delay sum: 2.53639, jitter sum: 1.52543
ECMP - delay sum: 42.2558, jitter sum: 1.27405
DRILL - delay sum: 4.45452, jitter sum: 1.61416
LetFlow - delay sum: 6.68186, jitter sum: 2.237


In [16]:
print("Small Flow Sums")
print("================")
print_delay_and_jitter_sums(df_spray_orig_small, "Packet Spray")
print_delay_and_jitter_sums(df_ecmp_orig_small, "ECMP")
print_delay_and_jitter_sums(df_drill_orig_small, "DRILL")
print_delay_and_jitter_sums(df_letflow_orig_small, "LetFlow")

Small Flow Sums
Packet Spray - delay sum: 2.5773642499999996, jitter sum: 0.38024074999999996
ECMP - delay sum: 6.560277900000001, jitter sum: 0.27974274999999993
DRILL - delay sum: 2.4171452, jitter sum: 0.16605
LetFlow - delay sum: 3.0214109999999996, jitter sum: 0.39288825
